## SpanBERT model training, contents include
#### 1. Training data preparation
    x: Input features are text (dataset_dict["input_ids"], dataset_dict["token_type_ids"], dataset_dict["attention_mask"]) and sentiment label
    y: Selected text/answer
#### 2. Model training
#### 3. Inference
**Note: refer to original post for detail https://www.kaggle.com/code/anasofiauzsoy/tweet-sentiment-extraction-with-tf2-spanbert/notebook**

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        if 'checkpoint' not in  filename:
            print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

kaggle/input/tweet-sentiment-extraction/test.csv
kaggle/input/tweet-sentiment-extraction/sample_submission.csv
kaggle/input/tweet-sentiment-extraction/train.csv
kaggle/input/spanbert-pt/spanbert-base-cased/config.json
kaggle/input/spanbert-pt/spanbert-base-cased/pytorch_model.bin
kaggle/input/spanbert-pt/spanbert-base-cased/vocab.txt
kaggle/input/spanbert-pt/spanbert-large-cased/config.json
kaggle/input/spanbert-pt/spanbert-large-cased/pytorch_model.bin
kaggle/input/spanbert-pt/spanbert-large-cased/vocab.txt


In [3]:
import matplotlib.pyplot as plt
import os
import re
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from transformers import BertTokenizer, TFBertModel, BertConfig, BertTokenizerFast 
print(tf.__version__)

2022-08-17 08:37:49.168812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-17 08:37:49.168857: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/cortex-installs/miniconda/envs/cortex-python3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.9.1


In [19]:
data = pd.read_csv("kaggle/input/tweet-sentiment-extraction/train.csv")
test = pd.read_csv("kaggle/input/tweet-sentiment-extraction/test.csv")
print('training size: {}; test size: {}'.format(data.shape, test.shape))

training size: (27481, 4); test size: (3534, 3)


In [20]:
display(data.head(3))
display(test.head(3))

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative


,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative


### Data Preprocessing

In [21]:
data = data[data.sentiment != "neutral"]
print(data.shape)
data = data[pd.notnull(data.selected_text)]
print(data.shape)

(16363, 4)
(16363, 4)


In [22]:
max_len = 384
tokenizer = BertTokenizerFast.from_pretrained("kaggle/input/spanbert-pt/spanbert-base-cased/")

In [24]:
class TrainTweetExample:
    def __init__(self, sentiment, text, start_char_idx, selected_text, all_answers):
        self.sentiment = sentiment
        self.text = text
        self.start_char_idx = start_char_idx
        self.selected_text = selected_text
        self.all_answers = all_answers
        self.skip = False

    def preprocess(self):
        text = self.text
        sentiment = self.sentiment
        selected_text = self.selected_text
        start_char_idx = self.start_char_idx

        # Clean text, sentiment, and answer
        sentiment = " ".join(str(sentiment).split())
        text = " ".join(str(text).split())
        answer = " ".join(str(selected_text).split())

        # Find end character index of answer in text
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(text):
            self.skip = True
            return

        # Mark the character indexes in text that are in answer
        is_char_in_ans = [0] * len(text)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize text
        tokenized_text = tokenizer.encode_plus(text, return_offsets_mapping=True, max_length = max_len)

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_text.offset_mapping):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize sentiment
        tokenized_sentiment = tokenizer.encode_plus(sentiment, return_offsets_mapping=True, max_length = max_len)

        # Create inputs
        input_ids = tokenized_text.input_ids + tokenized_sentiment.input_ids[1:]
        token_type_ids = [0] * len(tokenized_text.input_ids) + [1] * len(
            tokenized_sentiment.input_ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_text.offset_mapping

In [25]:
def create_tweet_examples(data):
    tweet_examples = []
    all_answers = data.selected_text.values
    count = 0
    for index, row in data.iterrows():
        sentiment = row['sentiment']
        text = row['text']
        selected_text = row['selected_text']

        try: ## turn the data into TrainTweetExample objects
            start_char_idx = row.text.index(row.selected_text.split()[0])
            tweet_eg = TrainTweetExample(
                sentiment, text, start_char_idx, selected_text, all_answers
            )
            tweet_eg.preprocess()
            tweet_examples.append(tweet_eg)
        except: ## keep track of the number that can't be tokenized/processed
            count += 1 
    print("Couldn't process",count,"points")
    return tweet_examples

In [39]:
class TestTweetExample:
    def __init__(self, sentiment, text):
        self.sentiment = sentiment
        self.text = text
        self.skip = False

    def preprocess(self):
        text = self.text
        sentiment = self.sentiment

        # Clean text, answer and sentiment
        sentiment = " ".join(str(sentiment).split())
        text = " ".join(str(text).split())

        # Tokenize text
        tokenized_text = tokenizer.encode_plus(text, return_offsets_mapping=True, max_length = max_len)

        # Tokenize sentiment
        tokenized_sentiment = tokenizer.encode_plus(sentiment, return_offsets_mapping=True, max_length = max_len)

        # Create inputs
        input_ids = tokenized_text.input_ids + tokenized_sentiment.input_ids[1:]
        token_type_ids = [0] * len(tokenized_text.input_ids) + [1] * len(
            tokenized_sentiment.input_ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.text_token_to_char = tokenized_text.offset_mapping
        self.start_token_idx = 0
        self.end_token_idx = 0

In [27]:
def create_tweet_examples_test(data):
    tweet_examples = []
    all_answers = None
    count = 0
    for index, row in data.iterrows():
        sentiment = row['sentiment']
        text = row['text']
        selected_text = None

        try: ## turn the data into TestTweetExample objects
            start_char_idx = 0
            tweet_eg = TestTweetExample(
                sentiment, text
            )
            tweet_eg.preprocess()
            tweet_examples.append(tweet_eg)
        except: ## keep track of the number that can't be tokenized/processed
            count += 1
    print("Couldn't process",count,"points")
    return tweet_examples

In [28]:
def create_inputs_targets(tweet_examples):
    """Create data inputs to SpanBERT model
    
    Return:
            x: list - input features
            y: list - response with length 2 (2 arrays: first array contains start token index and second array contains end token index)
    """
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in tweet_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y

In [29]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(data, test_size = 0.1)

In [31]:
train.head(3)

,textID,text,selected_text,sentiment
3091,80d01af3a9,ha... I thought you would enjoy the Family Gu...,bored,negative
19020,7883967f6f,Bacon Fail. Ended up with a commiseration muffin,Fail.,negative
22843,cd8deca773,Walmart opening its first store in Amritsar to...,hoping it,positive


In [30]:
train_tweet_examples = create_tweet_examples(train)
x_train, y_train = create_inputs_targets(train_tweet_examples)
print(f"{len(train_tweet_examples)} training points created.")

eval_tweet_examples = create_tweet_examples(validation)
x_eval, y_eval = create_inputs_targets(eval_tweet_examples)
print(f"{len(eval_tweet_examples)} evaluation points created.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Couldn't process 0 points
14726 training points created.
Couldn't process 0 points
1637 evaluation points created.


In [64]:
print(train.shape, len(train_tweet_examples))

(14726, 4) 14726


In [69]:
for i in range(3):
    print(train_tweet_examples[i].sentiment,
          train_tweet_examples[i].text,
          train_tweet_examples[i].input_ids,
          train_tweet_examples[i].token_type_ids,
          train_tweet_examples[i].attention_mask,
          train_tweet_examples[i].start_token_idx,
          train_tweet_examples[i].end_token_idx,
          train_tweet_examples[i].context_token_to_char)

negative  ha... I thought you would enjoy the Family Guy reference... and I`m more bored than you [101, 5871, 119, 119, 119, 178, 1354, 1128, 1156, 5548, 1103, 1266, 2564, 3835, 119, 119, 119, 1105, 178, 169, 182, 1167, 11920, 1190, 1128, 102, 4366, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [74]:
print(len(y_train), y_train[0].shape)

2 (9753,)


In [71]:
print(x_train[:1])
print(y_train[:2])

[array([[  101,  5871,   119, ...,     0,     0,     0],
       [  101, 20503,  8693, ...,     0,     0,     0],
       [  101, 20049, 16882, ...,     0,     0,     0],
       ...,
       [  101,   178,  1176, ...,     0,     0,     0],
       [  101,  6476,   113, ...,     0,     0,     0],
       [  101,  1139,  5919, ...,     0,     0,     0]])]
[array([22,  2, 25, ..., 24, 24,  3]), array([22,  3, 26, ..., 24, 25,  6])]


In [30]:
test_tweet_examples = create_tweet_examples_test(test)
x_test, _ = create_inputs_targets(test_tweet_examples)
print(f"{len(test_tweet_examples)} test points created.")

Couldn't process 0 points
3534 test points created.


### Creating & Training Model

In [35]:
def create_model():

    PATH = 'kaggle/input/spanbert-pt/spanbert-base-cased/'
    encoder = TFBertModel.from_pretrained(PATH, from_pt = True)
    
    ## QA Model
    input_ids = layers.Input(shape=(max_len ,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
         input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]
    

    start_logits = layers.Dropout(0.3)(embedding)
    start_logits = layers.Conv1D(128,2,padding='same')(start_logits)
    start_logits = layers.LeakyReLU()(start_logits)
    start_logits = layers.Conv1D(64,2,padding='same')(start_logits)
    start_logits = layers.Dense(1, name="start_logit")(start_logits)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dropout(0.3)(embedding)
    end_logits = layers.Conv1D(128,2,padding='same')(end_logits)
    end_logits = layers.LeakyReLU()(end_logits)
    end_logits = layers.Conv1D(64,2,padding='same')(end_logits)
    end_logits = layers.Dense(1, name="end_logit")(end_logits)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model

In [36]:
use_tpu = False
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

2022-08-16 17:40:41.611646: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-16 17:40:41.611687: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-16 17:40:41.611724: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-235-19-127.aws.lmig.com): /proc/driver/nvidia/version does not exist
2022-08-16 17:40:41.612008: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All PyTorch model weights were used when initializing TFBertModel.

Some weights or buffers o

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_3[0][0]',            

/opt/cortex-installs/miniconda/envs/cortex-python3/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [37]:
def normalize_text(text):
    text = text.lower()

    # Remove punctuations
    exclude = set(string.punctuation)
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text


class ExactMatch(keras.callbacks.Callback):
    """
    Each `TweetExample` object contains the character level offsets for each token
    in its input text. We use them to get back the span of text corresponding
    to the tokens between our predicted start and end tokens.
    All the ground-truth answers are also present in each `TweetExample` object.
    We calculate the percentage of data points where the span of text obtained
    from model predictions matches one of the ground-truth answers.
    """

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_tweet_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            tweet_eg = eval_examples_no_skip[idx]
            offsets = tweet_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = tweet_eg.text[pred_char_start:pred_char_end]
            else:
                pred_ans = tweet_eg.text[pred_char_start:]

            normalized_pred_ans = normalize_text(pred_ans)
            normalized_true_ans = [normalize_text(_) for _ in tweet_eg.all_answers]
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [38]:
batch_size = 8
div = len(x_train[0]) - (len(x_train[0]) % batch_size)
x = list(np.array(x_train)[:,:div]) ## inputs must be divisible by batch size 
y = list(np.array(y_train)[:,:div])

In [39]:
exact_match_callback = ExactMatch(x_eval, y_eval)
model.fit(
    list(np.array(x)),
    list(np.array(y)),
    epochs=1,
    verbose=1,
    batch_size=batch_size,
    callbacks=[exact_match_callback],
)

/opt/cortex-installs/miniconda/envs/cortex-python3/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


35/35 [==============================] - 500s 14s/step lo

epoch=1, exact match score=0.55
1218/1218 [==============================] - 13183s 11s/step - loss: 3.3840 - activation_loss: 1.5735 - activation_1_loss: 1.8105


### Generating Predictions

In [42]:
pred = model.predict(x_test)
np.array(pred).shape

111/111 [==============================] - 1600s 14s/step


(2, 3534, 384)

In [43]:
test['prediction'] = np.zeros(len(test.text.values))

In [45]:
pred_start, pred_end = pred
count = 0

pred_text = []
for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
    tweet_eg = test_tweet_examples[idx]
    if (tweet_eg.skip == True):
        pred_text.append("")
    else: 
        offsets = tweet_eg.text_token_to_char
        start = np.argmax(start)
        end = np.argmax(end)
        if start >= len(offsets):
            continue
        pred_char_start = offsets[start][0]
        if end < len(offsets):
            pred_char_end = offsets[end][1]
            pred_ans = tweet_eg.text[pred_char_start:pred_char_end + 1]
        else:
            pred_ans = tweet_eg.text[pred_char_start:]

        normalized_pred_ans = normalize_text(pred_ans)
        #print(normalized_pred_ans)
        pred_text.append(normalized_pred_ans)
        test['prediction'][idx] = normalized_pred_ans

/tmp/ipykernel_3496/815857967.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['prediction'][idx] = normalized_pred_ans


In [46]:
test.prediction[test.prediction == 0] = test.text[test.prediction == 0]
test.prediction[test.sentiment == "neutral"] = test.text[test.sentiment=="neutral"]
test.prediction[test.prediction ==''] = test.text[test.prediction =='']

def clean_text(text):
    words = str(text).split()
    words = [x for x in words if not x.startswith("http")]
    words = " ".join(words)
    return words

test['prediction'] = test['prediction'].apply(clean_text)

In [49]:
test[:10]

,textID,text,sentiment,prediction
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,Last session of the day
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,exciting
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,shame
3,01082688c6,happy bday!,positive,happy
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,like
5,726e501993,that`s great!! weee!! visitors!,positive,great
6,261932614e,I THINK EVERYONE HATES ME ON HERE lol,negative,hates
7,afa11da83f,"soooooo wish i could, but im in school and my...",negative,wish
8,e64208b4ef,and within a short time of the last clue all ...,neutral,and within a short time of the last clue all o...
9,37bcad24ca,What did you get? My day is alright.. haven`...,neutral,What did you get? My day is alright.. haven`t ...
